In [1]:
import sys
sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append("/home/webphy/Desktop/dnn_processor/dataset/") # just to enable `dataset`
seed = 0

import numpy as np
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import dataset
import plotting
import qkeras
import keras

2024-02-11 13:13:46.544966: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-11 13:13:46.569093: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 13:13:46.569115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 13:13:46.569826: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-11 13:13:46.574197: I tensorflow/core/platform/cpu_feature_guar

In [2]:
BATCH_SIZE = 64
train_images_rgx = "../../dataset/train_set_4x/*/*.jpeg"
valid_images_rgx = "../../dataset/test_set_4x/*/*.jpeg"

train_ds, val_ds, nclasses = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, BATCH_SIZE, 128
)

2024-02-11 13:13:47.877339: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-11 13:13:47.900229: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-11 13:13:47.900358: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
from qkeras.utils import load_qmodel
from qkeras.estimate import print_qstats

model_path = "qatwqkeras/saved_qmodels/qatwqkeras.h5"

qmodel = load_qmodel(
    model_path,
    compile=False,
)
qmodel.compile(
    "adam", loss=tf.keras.losses.CategoricalCrossentropy(True), metrics=["accuracy"]
)

score = qmodel.evaluate(val_ds)
print("Accuracy: {}".format(score[1]))

print_qstats(qmodel)

2024-02-11 13:13:51.970823: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


5/5 [==============================] - 3s 33ms/step - loss: 0.4763 - accuracy: 0.8736
Accuracy: 0.8735632300376892
Instructions for updating:
Use ref() instead.


/home/webphy/Desktop/dnn_processor_v2/.venv/lib/python3.10/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(
Instructions for updating:
Use ref() instead.



Number of operations in model:
    q_conv2d                      : 196608 (smult_8_8)
    q_conv2d_1                    : 129600 (smult_8_8)
    q_conv2d_2                    : 18000 (smult_8_8)
    q_dense                       : 5625  (smult_8_4)

Number of operation types in model:
    smult_8_4                     : 5625
    smult_8_8                     : 344208

Weight profiling:
    q_conv2d_weights               : 192   (8-bit unit)
    q_conv2d_bias                  : 4     (8-bit unit)
    q_conv2d_1_weights             : 576   (8-bit unit)
    q_conv2d_1_bias                : 16    (8-bit unit)
    q_conv2d_2_weights             : 80    (8-bit unit)
    q_conv2d_2_bias                : 5     (8-bit unit)
    q_dense_weights                : 5625  (8-bit unit)
    q_dense_bias                   : 5     (8-bit unit)

Weight sparsity:
... quantizing model
    q_conv2d                       : 0.0408
    q_conv2d_1                     : 0.0152
    q_conv2d_2                     

# extract input_activation, conv1, and output activation

In [4]:
x = x_in = keras.layers.Input(shape=(128,128,3))

x = qmodel.layers[1](x)
x = qmodel.layers[2](x)
x = qmodel.layers[3](x)

one_layer_model = keras.models.Model(x_in, x)
one_layer_model.summary()


one_layer_model.save(model_path.replace(".h5", "_one_layer.h5"))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 q_activation (QActivation)  (None, 128, 128, 3)       0         
                                                                 
 q_conv2d (QConv2D)          (None, 32, 32, 4)         196       
                                                                 
 q_activation_1 (QActivatio  (None, 32, 32, 4)         0         
 n)                                                              
                                                                 
 q_conv2d_1 (QConv2D)        (None, 15, 15, 16)        592       
                                                                 
 q_activation_2 (QActivatio  (None, 15, 15, 16)        0         
 n)                                                          

/home/webphy/Desktop/dnn_processor_v2/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
